In [ ]:
import numpy as np
from knn import KNN

class HyperparameterTuner:
    def __init__(self):
        self.best_k = None
        self.best_distance_function = None
        self.best_scaler = None
        self.best_model = None

    # TODO: find parameters with the best f1 score on validation dataset
    def tuning_without_scaling(self, distance_funcs, x_train, y_train, x_val, y_val):

        result_list = []        
        
        max_score = -99999999
                        
        for k in range(1,30,2):
            for i in range(5):
                dist_fcn = list(distance_funcs.values())[i]
                knn = KNN(k, dist_fcn)
                knn.train(x_train, y_train)
                pred_val = knn.predict(x_val)
                f_score = f1_score(y_val, pred_val)

                if f_score > max_score:
                    max_score = f_score
                    best_num = k            
                    best_dist_fcn = dist_fcn
                    best_dist_index = i
                    best_mod = knn                
                elif f_score == max_score: 
                    if best_dist_index > i:
                        max_score = f_score
                        best_num = k            
                        best_dist_fcn = dist_fcn
                        best_dist_index = i
                        best_mod = knn                
                    elif best_dist_index == i:
                        if k < best_num:
                            max_score = f_score
                            best_num = k            
                            best_dist_fcn = dist_fcn
                            best_dist_index = i
                            best_mod = knn     

        # You need to assign the final values to these variables
        self.best_k = None
        self.best_distance_function = None
        self.best_model = None

        self.best_k = best_num
        self.best_distance_function = list(distance_funcs.keys())[best_dist_index]
        self.best_model = best_mod
    # TODO: find parameters with the best f1 score on validation dataset, with normalized data
    def tuning_with_scaling(self, distance_funcs, scaling_classes, x_train, y_train, x_val, y_val):
        
        result_list = []        
        
        max_score = -99999999
        
        for scale in range(2):
            scaler = list(scaling_classes.values())[scale]()
            scaled_train = scaler(x_train)
            scaled_val = scaler(x_val)           
            
            for k in range(1,30,2):
                for i in range(5):
                    dist_fcn = list(distance_funcs.values())[i]
                    knn = KNN(k, dist_fcn)
                    knn.train(x_train, y_train)
                    pred_val = knn.predict(x_val)
                    f_score = f1_score(y_val, pred_val)

                    if f_score > max_score:
                        max_score = f_score
                        best_num = k            
                        best_dist_fcn = dist_fcn
                        best_dist_index = i
                        best_scale = scale
                        best_mod = knn                
                    elif f_score == max_score: 
                        if best_scale < scale:
                            max_score = f_score
                            best_num = k            
                            best_dist_fcn = dist_fcn
                            best_dist_index = i
                            best_scale = scale
                            best_mod = knn                                            
                        elif best_scale > scale:                                                
                            if best_dist_index > i:
                                max_score = f_score
                                best_num = k            
                                best_dist_fcn = dist_fcn
                                best_dist_index = i
                                best_scale = scale
                                best_mod = knn                
                            elif best_dist_index == i:
                                if k < best_num:
                                    max_score = f_score
                                    best_num = k            
                                    best_dist_fcn = dist_fcn
                                    best_dist_index = i
                                    best_scale = scale
                                    best_mod = knn     

        # You need to assign the final values to these variables
        self.best_k = None
        self.best_distance_function = None
        self.best_scaler = None
        self.best_model = None
        
        self.best_k = best_num
        self.best_distance_function = list(distance_funcs.keys())[best_dist_index]
        self.best_scaler = list(scaling_classes.keys())[best_scale]
        self.best_model = best_mod

In [ ]:
class NormalizationScaler:
    def __init__(self):
        pass

    # TODO: normalize data
    def __call__(self, features):
        
        features_norm = []

        for point in features:        
            if np.linalg.norm(point) == 0:
                features_norm.append(point)
            elif np.linalg.norm(point) != 0:
                features_norm.append(list(point/np.linalg.norm(point)))
        
        return features_norm
        
        # raise NotImplementedError


class MinMaxScaler:
    
    def __init__(self):
        self.x_min=[]
        self.x_max=[]

    def __call__(self, features):
    
        if  self.x_max == []:                
            self.x_max = np.max(features,axis = 0)
        if self.x_min == []:
            self.x_min = np.min(features,axis = 0)

        min_max = features

        for i in range(len(features)):
            for k in range(len(features[0])):
                if self.x_min[k] != self.x_max[k]:                                                
                    min_max[i][k] = (min_max[i][k] - self.x_min[k]) / (self.x_max[k] - self.x_min[k])
                else:                        
                    min_max[i][k] = (min_max[i][k] - self.x_min[k]) / len(features[0])

        return min_max
    
scaling_classes = {
    'min_max_scale': MinMaxScaler,
    'normalize': NormalizationScaler,
}

In [ ]:
features = [[2, -1], [-1, 5], [0, 0]]

for index, key in scaling_classes.items():    
    if index == "min_max_scale": print("this is minmax")
    elif index != "min_max_scale": print("not minmax")
    
    
dir(scaling_classes.items())
# for s in range(2):
#     print(list(scaling_classes.values())[s]())